In [2]:
import numpy as np
import numpy.linalg as la
import sympy as sp

In [39]:
def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())

def calculated_jacobian(fs, variables, x_k):
    '''
    fs need to EQUAL TO ZERO!
    '''
    size = len(fs)
    result = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            result[i][j] = subs_all(sp.diff(fs[i], variables[j]), variables, x_k)
    return result

def Newton_nd_step(x_prev, f_strs, var_str):
    '''
    Given x_k, string of functions equal to 0, and string for
    variables, calculate the parameter for the next iteration
    of Newton's method for solving ND non-linear system of equations
    '''
    fs = [sp.sympify(f_str) for f_str in f_strs]
    variables = sp.symbols(var_str)
    jac = calculated_jacobian(fs, variables, x_prev)
    f_vals = np.array([subs_all(f, variables, x_prev) for f in fs])
    s = la.solve(jac, -1*f_vals)
    return jac, x_prev+s

In [47]:
Newton_nd_step([1, -2], 
               ['(-1)*x**2+(0)*x+(-2)+(2)*y**2+(-1)*y+(3)*x*y',
                '(-2)*x**2+(1)*x+(1)+(3)*y**2+(0)*y+(-3)*x*y'],
               'x y')

(array([[ -8.,  -6.],
        [  3., -15.]]),
 array([ 0.32608696, -0.93478261]))

In [22]:
# 15.2

def Bisection_n_step(f_str, var_str, bound, step_num):
    f = sp.sympify(f_str)
    variable = sp.symbols(var_str)
    for i in range(step_num):
        value = [f.subs(variable, value) for value in bound]
        mid_point = sum(bound)/2
        mid_value = f.subs(variable, mid_point)
        if mid_value * value[0] > 0:
            bound[0] = mid_point
            continue
        else:
            bound[1] = mid_point
    return bound


In [35]:
Bisection_n_step(f_str = 'sin(x/6)',
                 var_str = 'x',
                 bound = [-2, 4],
                 step_num = 3)

[-0.5, 0.25]

In [7]:
def Newton_1D_step_aux(x_prev, f, df, var):
    '''
    Given x_k, function, differentiation of the function, and the variable
    Return the numerical value of x_{k+1}
    Operate as a lighter weighted version of Newton_1D_step
    '''
    return float(x_prev - f.subs(var, x_prev) / df.subs(var, x_prev))

def Newton_1D(x_0, f_str, var_str, err_bound):
    '''
    Given initial value x_0, string representation of function, string representation of variable
    as well as the error bound
    Return the number of iterations it takes to get to the error bound
    And the final value
    '''
    f = sp.sympify(f_str)
    var = sp.symbols(var_str)
    df = sp.diff(f, var)
    x_curr = x_0
    iterations = 0
    while abs(float(f.subs(var, x_curr))) > err_bound:
        x_curr = Newton_1D_step_aux(x_curr, f, df, var)
        iterations += 1
    return iterations, x_curr

In [8]:
err_bound = 10**-5
Newton_1D(3.5, 'x*E**x', 'x', err_bound)

(9, 1.1858764201470775e-09)

In [36]:
x_curr = 2
f = sp.sympify('sin(x/6)')
var = sp.symbols('x')
df = sp.diff(f, var)
Newton_1D_step_aux(x_curr, f, df, var)

-0.07752129706345294

In [50]:
def Secant_n_step(f_str, var_str, x0, x1, step_num):
    f = sp.sympify(f_str)
    variable = sp.symbols(var_str)
    xold = x1
    xoold = x0
    for i in range(step_num):
        slope = (f.subs(variable, xold) - f.subs(variable, xoold))/(x1-x0)
        xnew = xold - f.subs(variable, xold)/slope
        xoold = xold
        xold = xnew
    return float(xnew.evalf())

In [51]:
Secant_n_step(f_str = 'sin(x/6)', 
              var_str = 'x', 
              x0 = 1,
              x1 = 2, 
              step_num = 1)

-0.028503468808001618

In [ ]:
def Bisection_Method_for_Root_Finding_in_1D(function, intervals, epsilon, n_iter):
    roots = []
    for interval in intervals:
        xl, xr = interval
        for i in range(n_iter):
            yl, yr = function(xl), function(xr)
            if yl * yr >= 0:
                roots.append(None)
                break
            xm = (xl + xr) / 2
            ym = function(xm)
            if abs(ym) < epsilon:
                roots.append(xm)
                break
            elif yl * ym > 0:
                xl = xm
            elif yr * ym > 0:
                xr = xm
            continue
    return roots
            

roots = Bisection_Method_for_Root_Finding_in_1D(function, intervals, epsilon, n_iter)

In [ ]:
import numpy as np

def Implementing_Secant_Method_for_1D_Problem(f, xks):
    roots = []
    x0, x1 = xks
    x = x1
    for i in range(5):
        x -= f(x) / ((f(x1) - f(x0)) / (x1 - x0))
        x0, x1 = x1, x
        roots.append(x)
    return np.array(roots)

roots = Implementing_Secant_Method_for_1D_Problem(f, xks)

In [ ]:
import numpy as np
import numpy.linalg as la
import sympy as sp


def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())

def calculated_jacobian(fs, variables, x_k):
    '''
    fs need to EQUAL TO ZERO!
    '''
    size = len(fs)
    result = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            result[i][j] = subs_all(sp.diff(fs[i], variables[j]), variables, x_k)
    return result

def Newton_nd_step(x_prev, f_strs, var_str):
    '''
    Given x_k, string of functions equal to 0, and string for
    variables, calculate the parameter for the next iteration
    of Newton's method for solving ND non-linear system of equations
    '''
    fs = [sp.sympify(f_str) for f_str in f_strs]
    variables = sp.symbols(var_str)
    jac = calculated_jacobian(fs, variables, x_prev)
    f_vals = np.array([subs_all(f, variables, x_prev) for f in fs])
    s = la.solve(jac, -1*f_vals)
    res = la.norm(f_vals)
    return x_prev+s, res
    
    
f_strs = ['x**3 - y**2', 'x+y*x**2 - 2']
var_str = 'x y'
res = np.inf
root = xi

while res > tol:
    root, res = Newton_nd_step(root, f_strs, var_str)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

iterations_to_converge = []
Xs = np.zeros((len(X0), 10))
iteration_limit = 100

def f(x):
    return 0.5 * x * np.sin(2 * x) - 0.8 * x + 1

def dx(x):
    return 0.5 * np.sin(2 * x) + 0.5 * x * 2 * np.cos(2 * x) - 0.8
    
    
for j in range(len(X0)):
    x_old = X0[j]
    x_new = np.inf
    for i in range(1, iteration_limit+1):
        x_new = x_old - f(x_old)/dx(x_old)
        if i <= 10:
            Xs[j][i-1] = x_new
        if abs(x_old-x_new) < tol or i == iteration_limit:
            iterations_to_converge.append(i)
            break
        x_old = x_new

# Use the following code to plot the result.
plt.figure(figsize = (8,6))
plot_x = np.linspace(-5, 5, 500)
plt.plot(plot_x, f(plot_x), label = "fx = 0.5x * sin(2x) - 0.8 * x + 1")
plt.plot(plot_x, [0 for i in range(len(plot_x))], label="f(x) = 0")
plt.scatter(X0, f(X0), c = [1 - t/100 for t in iterations_to_converge], label = "heat-map of times to converge")
for i in range(len(X0)):
    plt.text(X0[i], f(X0[i])+ (-1)**(i+1) * 0.5, str(iterations_to_converge[i]), ha="center", va="center", color="black")
plt.legend()
plt.title("Local convergence of Newton's method")
plt.show()